## Automação de Indicadores

In [ ]:
# Importando os dados 
# Criar uma tabela para cada loja
# Criar um pasta para cada loja e salvar as planilhas na pasta Backup Arquivos Lojas. O formato do arquivo salvo deve ser por ex: 05_23_Nome da loja.xlsx 
# Calcular os indicadores para cada loja
# Montar a OnePage
# Enviar pora cada reespectivo gerente
# Criar um ranking para as lojas 
# Enviar um e-mail para a diretoria

In [ ]:
#importando as bibliotecas
import pandas as pd
import os


### Parte 1 - Criação das tabelas da loja

In [ ]:
#importando base de dados lojas
tabela_lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='ISO-8859-1', sep=';')
tabela_lojas.head()

In [ ]:
#importando base de dados Emails
tabela_emails = pd.read_excel('Bases de Dados/Emails.xlsx')
tabela_emails.head()

In [ ]:
#importando base de dados Vendas
tabela_vendas = pd.read_excel('Bases de Dados/Vendas.xlsx')
tabela_vendas.head()

In [ ]:
# Criando uma nova tabela de Vendas contendo o nome da loja
tabela_vendas_completa = tabela_lojas.merge(tabela_vendas, how='inner', on=['ID Loja'])
tabela_vendas_completa

### Parte 2 - Criando um pasta, uma a tabela para cada loja e salvando os respectivos arquivos backup

In [ ]:
# criando uma lista das lojas 
lista_lojas = list(tabela_lojas['Loja'])

In [ ]:
# Criando uma pasta para cada loja
for loja in lista_lojas:
    
    if not os.path.isdir(f"Backup Arquivos Lojas\\{loja}"):
        os.mkdir(f'Backup Arquivos Lojas\\{loja}')


In [ ]:
# Criando uma tabela para cada loja

dicionario_lojas = {}

for loja in lista_lojas:
    dicionario_lojas[loja] = tabela_vendas_completa.loc[tabela_vendas_completa['Loja'] == loja]


In [ ]:
# Definindo a data do último dia 
data_atual = tabela_vendas_completa['Data'].max().strftime("%d/%m/%Y")
dia_atual = data_atual[:2]
mes_atual = data_atual[3:5]

In [ ]:
# Salvando os arquivos de backup nas respectivas 
for loja in dicionario_lojas.keys():

    # Verificando se a pasta já existe
    if not os.path.isdir(f"Backup Arquivos Lojas\\{loja}"):

        os.mkdir(f'Backup Arquivos Lojas\\{loja}')

    # Salvando o arquivos de da respectiva pasta
    dicionario_lojas[loja].to_excel(f'Backup Arquivos Lojas/{loja}/{mes_atual}_{dia_atual}_{loja}.xlsx', index=False)


### Parte 3 - Cálculo dos indicadores

#### Calculo do Faturamento -> Meta Ano: 1.650.000 / Meta Dia: 1000

In [ ]:
# Faturamento anual por loja
for loja in lista_lojas:
    faturamento_ano = dicionario_lojas[loja]
    faturamento_ano = faturamento_ano.groupby(['Loja']).sum(['Valor Final'])
    display(faturamento_ano['Valor Final'])

In [ ]:
# Faturamento hoje por loja
faturamento_por_loja = tabela_vendas_completa.groupby(['Data','Loja']).sum()['Valor Final'].agg(hoje)
melhor_loja_hoje = faturamento_por_loja.sort_values(axis=0,ascending=False)
print(f"A melhor loja hoje foi a loja {melhor_loja_hoje.index[0]}")


#### Calculo da Diversidade de Produtos (quantos produtos diferentes foram vendidos naquele período) -> Meta Ano: 120 / Meta Dia: 4

In [ ]:
tabela_vendas.groupby(['Produto']).sum(['Quantidade'])['Quantidade'].count()


tabela_vendas.groupby(['Produto','Data']).sum(['Quantidade'])['Quantidade'].count().unique()


####  Calculo do Ticket Médio por Venda -> Meta Ano: 500 / Meta Dia: 500

In [ ]:
# Ticket médio por ano
total_vendas = tabela_vendas['Quantidade'].count()

ticket_medio = faturamento_total / total_vendas
ticket_medio


In [ ]:
# Ticket médio por dia

